# KoNLPy
- 한글 자연어 처리를 쉽고 간결하게 처리할 수 있도록 만들어진 오픈소스 라이브러리
- 어절 단위에 대한 토크나이징은 NLTK로 충분히 해결할 수 있음. 여기서는 형태소 단위 위주
- Java 1.7 이상 + 환경변수 설정 필요

## 형태소 분석 및 품사 태깅
형태소[명사]: 의미를 가지는 가장 작은 단위.  
KoNLPy에 포함된 형태소 분석기:
- Hannanum
- Kkma
- Komoran
- Mecab
- Okt

In [52]:
from konlpy.tag import Okt
okt = Okt()

In [29]:
help(okt)

Help on Okt in module konlpy.tag._okt object:

class Okt(builtins.object)
 |  Okt(jvmpath=None, max_heap_size=1024)
 |  
 |  Wrapper for `Open Korean Text <https://github.com/open-korean-text/open-korean-text>`_.
 |  
 |  Open Korean Text is an open source Korean tokenizer written in Scala,
 |  developed by Will Hohyon Ryu.
 |  
 |  .. code-block:: python
 |  
 |      >>> from konlpy.tag import Okt
 |      >>> okt = Okt()
 |      >>> print(okt.morphs(u'단독입찰보다 복수입찰의 경우'))
 |      ['단독', '입찰', '보다', '복수', '입찰', '의', '경우']
 |      >>> print(okt.nouns(u'유일하게 항공기 체계 종합개발 경험을 갖고 있는 KAI는'))
 |      ['항공기', '체계', '종합', '개발', '경험']
 |      >>> print(okt.phrases(u'날카로운 분석과 신뢰감 있는 진행으로'))
 |      ['날카로운 분석', '날카로운 분석과 신뢰감', '날카로운 분석과 신뢰감 있는 진행', '분석', '신뢰', '진행']
 |      >>> print(okt.pos(u'이것도 되나욬ㅋㅋ'))
 |      [('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되나욬', 'Noun'), ('ㅋㅋ', 'KoreanParticle')]
 |      >>> print(okt.pos(u'이것도 되나욬ㅋㅋ', norm=True))
 |      [('이', 'Determiner'), ('것', 'Noun

- `okt.morphs(phrase, norm = False, stem = False)`
    - phrase를 형태소 단위로 나눈다. 
    - norm: 문장을 정규화할지 여부
    - stem: 각 단어에서 어간을 추출할지 여부

- `okt.nouns(phrase)`
    - phrase에서 명사만 뽑아낸다.

- `okt.phrases(phrase)`
    - phrase에서 어절을 뽑아낸다.

- `okt.pos(phrase, norm=False, stem=False, join=False)`
    - 각 품사를 태깅하는 역할. 주어진 텍스트를 형태소 단위로 나누고, 나눠진 각 형태소를 그에 해당하는 품사와 함게 리스트화하는 것.
    - join: 나눠진 형태소와 품사를 '형태소/품사' 형태로 같이 붙여서 리스트화.

In [53]:
text = '한글 자연어 처리는 재밌다 이제부터 열심히 해야지ㅎㅎㅎ'

print('morphs(): ',okt.morphs(text))
print('morphs(norm=True): ',okt.morphs(text, norm=True))
print('morphs(stem=True): ',okt.morphs(text, stem=True))
print('nouns(): ',okt.nouns(text))
print('phrases(): ',okt.phrases(text))
print('pos(): ',okt.pos(text))
print('pos(join=True): ',okt.pos(text, join=True))

morphs():  ['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '해야지', 'ㅎㅎㅎ']
morphs(norm=True):  ['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '해야지', 'ㅎㅎㅎ']
morphs(stem=True):  ['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '하다', 'ㅎㅎㅎ']
nouns():  ['한글', '자연어', '처리', '이제']
phrases():  ['한글', '한글 자연어', '한글 자연어 처리', '이제', '자연어', '처리']
pos():  [('한글', 'Noun'), ('자연어', 'Noun'), ('처리', 'Noun'), ('는', 'Josa'), ('재밌다', 'Adjective'), ('이제', 'Noun'), ('부터', 'Josa'), ('열심히', 'Adverb'), ('해야지', 'Verb'), ('ㅎㅎㅎ', 'KoreanParticle')]
pos(join=True):  ['한글/Noun', '자연어/Noun', '처리/Noun', '는/Josa', '재밌다/Adjective', '이제/Noun', '부터/Josa', '열심히/Adverb', '해야지/Verb', 'ㅎㅎㅎ/KoreanParticle']


In [54]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

train_data = pd.read_csv('./nsmc-master/ratings_train.txt', header = 0, delimiter = '\t', quoting = 3)

In [25]:
train_data['document'][:5]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

첫 번째 리뷰로 전처리 연습.  
1. 한글 문자가 아닌 것들을 모두 제거(by 정규표현식)

In [55]:
review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', train_data['document'][1])
print(review_text)

흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나


2. 불용어를 제거하기 위해 문장 단어로 나누기 + 형태소 분석기를 사용해 어간이 추출된 단어로 나누기

In [62]:
# 맞춤법과 문법 오류가 없는데 분석이 안되는 문장.
temp = '당신은요?'
temp_okt = okt.pos(temp, join=True)
print(temp_okt)

['당/Modifier', '신/Modifier', '은/Noun', '요/Josa', '?/Punctuation']


In [56]:
okt = Okt()
review_text = okt.morphs(review_text, stem = True)
print(review_text)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']
